In [3]:
import os
import sys
sys.path.append("..")
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import pandas as pd
from utils import *
import string
from collections import Counter
import spacy
import nltk
from nltk.tokenize import word_tokenize
import importlib.metadata as importlib_metadata
importlib_metadata.version('tqdm')
import spacy
from spacy import Language
nlp = spacy.load("de_core_news_sm")
import sqldf

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jannis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('tagsets')

## Actual Code to do initial POS-tagging!

In [ ]:
words_rem = df.text.map(remove_stopwords)
punct_rem = words_rem.map(lambda x:x.translate(str.maketrans('','',string.punctuation)))
pos_list=[]
for sentence in punct_rem:
    sentence_pos = get_sentence_pos_list(sentence,nlp)
    pos_list.append(sentence_pos)
pos_seqings = list(map(lambda x: ' '.join(x),pos_list))
df['pos_seq'] = pos_seqings
df['pos_list'] = pos_list

## Determining the POS-Union between True / False 

In [ ]:
df = pd.read_csv('../data/data_files/all_sentences_w_pos.csv')

In [ ]:
# define a custom aggregation function to calculate the pos_union value
def pos_union_func(x):
    # count the number of unique labels for this pos_seq
    label_count = len(set(x['label']))
    # if there is only one unique label, return True
    if label_count == 1:
        return False
    if label_count == 2:
        return True
    

# group by pos_seq and apply the custom aggregation function to calculate pos_union
pos_union = df.groupby('pos_seq').apply(pos_union_func).reset_index(name='pos_union')

# merge the pos_union back into the original dataframe
df_union = df.merge(pos_union, on='pos_seq', how='left')

# print the resulting dataframe
print(df_union)

In [ ]:
df_union.head()

In [4]:
df_union.to_csv('../data/data_files/all_sentences_w_pos_unions.csv')

NameError: name 'df_union' is not defined

In [5]:
df_union = pd.read_csv('../data/data_files/all_sentences_w_pos_unions.csv')

In [7]:
df_union.value_counts('pos_union')

pos_union
False    390007
True      73658
dtype: int64

In [6]:
df_union.value_counts('label')

label
False    353068
True     110677
dtype: int64

## Filter for pos_union & not Label

In [8]:
filtered_df = df_union.query('not (label == False and pos_union == True)')

In [9]:
filtered_df.value_counts('pos_union')

pos_union
False    390007
True      15963
dtype: int64

In [10]:
filtered_df.value_counts('label')

label
False    295373
True     110677
dtype: int64

In [11]:
filtered_df.to_csv('../data/data_files/all_sentences_posfilter_standard.csv')

## Some insights into what is filtered out:


In [13]:
filtered_out = df_union.query('(label == False and pos_union == True)')

In [14]:
print(filtered_out['text'])

6         Er war einer der 24 Angeklagten im Nürnberger ...
9                        Die Familie Heß stammte aus Böhmen
15                                  Seine Mutter Clara (geb
16        Münch) entstammte ebenfalls einer fränkischen ...
18                       Daher lernte er auch kein Englisch
                                ...                        
463720             Die Beschneidungsrate ist stark gesunken
463721    Auch in Australien sinkt die Beschneidungsrate...
463724        In Israel ist die Beschneidungsrate sehr hoch
463728    Japan hat eine der niedrigsten Beschneidungsra...
463738       Die Beschneidung solle einen Inzest verhindern
Name: text, Length: 57695, dtype: object


## Other stuff

In [ ]:
regex = r"[^A-Za-z0-9\s'':;.?()\[\]@\\\-/\$´€`&]+"
df_nan = df_union.copy()
df_nan['text'] = df_union['text'].fillna('')
df_nan = df_nan.reset_index(drop=True)
df_filtered = df_nan[~df_nan['text'].str.contains(regex)]
